In [1]:
import weaviate

In [7]:
import json
import os

# Read the aquarium schema JSON file
schema_path = os.path.join("..", "data", "demo-schema.json")
with open(schema_path, "r") as f:
    aquarium_schema = json.load(f)

In [8]:
aquarium_schema

{'classes': [{'class': 'Aquarium',
   'description': 'An aquarium containing fish.',
   'vectorIndexType': 'hnsw',
   'vectorizer': 'text2vec-transformers',
   'properties': [{'name': 'name',
     'dataType': ['text'],
     'description': 'Name of the aquarium.'},
    {'name': 'location',
     'dataType': ['text'],
     'description': 'Location of the aquarium.'},
    {'name': 'waterVolume',
     'dataType': ['number'],
     'description': 'Water volume of the aquarium in liters.'},
    {'name': 'hasMarineFish',
     'dataType': ['boolean'],
     'description': 'Does the aquarium have marine fish?'}]},
  {'class': 'Aquarist',
   'description': 'A person who takes care of aquariums.',
   'vectorIndexType': 'hnsw',
   'vectorizer': 'text2vec-transformers',
   'properties': [{'name': 'name',
     'dataType': ['text'],
     'description': 'Name of the aquarist.'},
    {'name': 'yearsExperience',
     'dataType': ['int'],
     'description': 'Years of experience as an aquarist.'}]}]}

In [10]:
import requests

url = "http://localhost:8080/v1/schema"

payload = aquarium_schema['classes'][0]

response = requests.post(url, json=payload)

print(response.json())

{'class': 'Aquarium', 'description': 'An aquarium containing fish.', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'moduleConfig': {'text2vec-transformers': {'poolingStrategy': 'masked_mean', 'vectorizeClassName': True}}, 'multiTenancyConfig': {'autoTenantActivation': False, 'autoTenantCreation': False, 'enabled': False}, 'properties': [{'dataType': ['text'], 'description': 'Name of the aquarium.', 'indexFilterable': True, 'indexRangeFilters': False, 'indexSearchable': True, 'moduleConfig': {'text2vec-transformers': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'name', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'Location of the aquarium.', 'indexFilterable': True, 'indexRangeFilters': False, 'indexSearchable': True, 'moduleConfig': {'text2vec-transformers': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'location', 'tokenization': 'word